In [3]:
#Import packages
import numpy as np
import pandas as pd
import math
import sklearn
import sklearn.preprocessing
import datetime
import os
import matplotlib.pyplot as plt
from sklearn.linear_model import HuberRegressor
import warnings
warnings.simplefilter(action='ignore', category=Warning)
import seaborn as sns
sns.set()
pd.options.mode.chained_assignment = None  # default='warn'

#Program that executes the rolling window train-val-test split 
#%run TimeBasedCV.ipynb

In [ ]:
df = pd.read_csv(r"C:\Users\krist\Documents\Data\ger_factor_data_from2003.csv", dtype ={"comp_tpci": str}, parse_dates =["eom"])

In [4]:
#Read in the cleansed data
df = pd.read_csv(r"C:\Users\nanye\Desktop\datashare\datashare.csv")

#Convert to float 32 (format needed for the most ML models)
df.head()

,permno,DATE,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,...,stdcf,ms,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2
0,10006,19570131,82249.000,1.122846,1.260784,0.047180,9.569953,0.025742,0.046433,0.044843,...,NaN,NaN,0.013234,9.411565e-08,0.015453,0.008058,0.355638,0.460420,1.120996e-07,37.0
1,10014,19570131,3903.375,0.426734,0.182102,-0.275641,6.237836,0.072103,0.046433,-0.086957,...,NaN,NaN,0.033305,6.610609e-06,0.047619,0.033495,1.152126,1.169610,9.229146e-08,NaN
2,10022,19570131,9273.250,1.066449,1.137313,-0.025490,7.008844,0.027648,0.046433,-0.060377,...,NaN,NaN,0.016023,2.286832e-06,0.020833,0.015589,0.815777,0.679803,1.181757e-07,NaN
3,10030,19570131,54465.875,0.926038,0.857547,0.018171,9.825337,0.021700,0.046433,0.044633,...,NaN,NaN,0.015295,1.464273e-07,0.039326,0.015849,0.739302,1.333656,6.126699e-08,NaN
4,10057,19570131,40250.000,1.247748,1.556875,0.025785,7.901007,0.025506,0.046433,0.086667,...,NaN,NaN,0.005954,1.380375e-06,0.056856,0.019945,0.755510,0.410391,3.315790e+00,NaN


In [5]:
df

,permno,DATE,mvel1,beta,betasq,chmom,dolvol,idiovol,indmom,mom1m,...,stdcf,ms,baspread,ill,maxret,retvol,std_dolvol,std_turn,zerotrade,sic2
0,10006,19570131,8.224900e+04,1.122846,1.260784,0.047180,9.569953,0.025742,0.046433,0.044843,...,NaN,NaN,0.013234,9.411565e-08,0.015453,0.008058,0.355638,0.460420,1.120996e-07,37.0
1,10014,19570131,3.903375e+03,0.426734,0.182102,-0.275641,6.237836,0.072103,0.046433,-0.086957,...,NaN,NaN,0.033305,6.610609e-06,0.047619,0.033495,1.152126,1.169610,9.229146e-08,NaN
2,10022,19570131,9.273250e+03,1.066449,1.137313,-0.025490,7.008844,0.027648,0.046433,-0.060377,...,NaN,NaN,0.016023,2.286832e-06,0.020833,0.015589,0.815777,0.679803,1.181757e-07,NaN
3,10030,19570131,5.446588e+04,0.926038,0.857547,0.018171,9.825337,0.021700,0.046433,0.044633,...,NaN,NaN,0.015295,1.464273e-07,0.039326,0.015849,0.739302,1.333656,6.126699e-08,NaN
4,10057,19570131,4.025000e+04,1.247748,1.556875,0.025785,7.901007,0.025506,0.046433,0.086667,...,NaN,NaN,0.005954,1.380375e-06,0.056856,0.019945,0.755510,0.410391,3.315790e+00,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4117295,93423,20211231,3.144398e+06,1.923076,3.698221,-0.673385,16.305232,0.062619,0.491455,-0.110868,...,0.960494,3.0,0.043829,3.698841e-10,0.073435,0.029678,0.399725,9.215947,5.031856e-09,79.0
4117296,93426,20211231,4.326610e+05,1.224523,1.499457,-0.061462,12.419552,0.033723,0.346308,0.007040,...,0.050973,6.0,0.025875,1.097624e-08,0.046055,0.019302,0.481331,1.649750,3.023395e-08,36.0
4117297,93427,20211231,4.092710e+06,0.887083,0.786917,-0.080295,14.884834,0.045395,0.346308,0.151667,...,0.052512,6.0,0.029966,8.593845e-10,0.131563,0.032861,0.440057,2.444487,2.224496e-08,36.0
4117298,93434,20211231,1.130478e+05,0.512942,0.263110,-0.543760,12.243872,0.088124,0.270005,-0.319347,...,0.556876,2.0,0.069943,1.145228e-07,0.036932,0.035858,0.968425,4.557546,3.155353e-08,1.0


Rank standardization in inreval[-1. 1]

A rank standardization procedure following Kelly, Pruitt, and Su (2019) and Freyberger, Neuhierl and Weber(2020) is employed.
The stock characteristics are ranked month-by-month cross-sectionally and the ranks are mapped into the [-1, 1], thereby transforming features into a uniform distribution and increasing the insensitivity to outriers.

In [ ]:
#Standardizze all independent variables
features = df.columns[~df.columns.isin([])]

#Ranks in [0,1] interval
df[features]=df.groupby()[features].rank(pct=True)

#multiply by 2 and substact 1 to get ranks in interval[-1, 1]
df[features] = 2*df[features] - 1

All Firms

In [ ]:
# Define the splitting scheme:
# Rolling window  train-val-test split
# The rolling window is set to 96 months (8 years)
# 5 years for training, 2 years for validation, and 1 year for out-of-samlpe testing
tcv = TimeBasedCV(
    
)